In [243]:
import pandas as pd
import glob

import sys

sys.path.append('..')
from git_repo_crawler import data_handler

In [244]:
import pandas as pd
import glob

import sys

sys.path.append('..')
from git_repo_crawler import data_handler

In [245]:
datapath = "../data/raw"
df = data_handler.load_all(datapath)

In [246]:
datecols = [c for c in df.columns if "date" in c.lower()]
for c in datecols:
    df[c] = pd.to_datetime(df[c])
    
df.dtypes

hash                               object
msg                                object
author_date           datetime64[ns, UTC]
author_timezone                     int64
committer_date        datetime64[ns, UTC]
committer_timezone                  int64
branches                           object
in_main_branch                       bool
merge                                bool
parents                            object
project_name                       object
project_path                      float64
data_source                        object
author_name                        object
author_email                       object
committer_name                     object
committer_email                    object
reference                          object
fpath                              object
dtype: object

In [247]:
# drop rows for hits in db and dev/db /*
df = pd.DataFrame(df[~df['fpath'].str.split('/').apply(lambda x: "db" in x)])

In [248]:
import re
# figure out what kind of reference each row is
df['reftype'] = df['reference'].apply(lambda x: re.findall(r"[\w']+", x)[0]).str.upper()

In [249]:
# clean up timezones
for role in ['author','committer']:
    k1 = f"{role}_date"
    k2 = f"{role}_timezone"
    
    df[k1] = df[k1] + df[k2].apply(lambda x: timedelta(seconds=x))
    df = df.drop(columns=[k2,])

# now that our times are normalized, find the earliest for each row
df['min_date'] = df[['author_date','committer_date']].min(axis=1)

# reorder columns
cols = ['reference', 'min_date','reftype', 'fpath', 'project_name', 'project_path', 
        'data_source', 'hash', 'msg', 'branches', 'in_main_branch', 'merge', 'parents',
        'author_name', 'author_email', 'author_date',
        'committer_name', 'committer_email',  'committer_date', 
       ]
df = df[cols]

# get rid of stuff we don't need
drop_cols = ['project_path','in_main_branch','merge','branches','parents', ]
df = df.drop(columns=drop_cols)

# we want to consolidate all the files changed in a commit for a vul
agg = {k: "first" for k in df.columns}
agg['fpath'] = list
df = df.groupby(['reference','hash']).aggregate(agg).reset_index(drop=True)

# sort ascending by date
df = df.sort_values(by='min_date',ascending=True)
# then drop duplicates for the same vul 
df = df.drop_duplicates(subset=["reference","project_name"],keep='first')
df['fpath'] = df['fpath'].apply(set).apply(list)

df

,reference,min_date,reftype,fpath,project_name,data_source,hash,msg,author_name,author_email,author_date,committer_name,committer_email,committer_date
2217,CVE-2011-0364,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/17155.py],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2335,CVE-2011-3167,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/18388.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2336,CVE-2011-3175,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/19958.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2337,CVE-2011-3176,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/19959.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2338,CVE-2011-3192,2013-12-03 19:44:07+00:00,CVE,[platforms/linux/dos/18221.c],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6834,CVE-2020-11749,2020-07-27 05:02:04+00:00,CVE,[exploits/php/webapps/48707.txt],exploitdb,https://github.com/offensive-security/exploitd...,e46d9f65ff00b5da8af2bf715f59767ae265523c,DB: 2020-07-27\n\n32 changes to exploits/shell...,Offensive Security,info@exploit-db.com,2020-07-27 05:02:04+00:00,Offensive Security,info@exploit-db.com,2020-07-27 05:02:04+00:00
6488,CVE-2019-16116,2020-07-28 05:01:59+00:00,CVE,[exploits/windows/remote/48657.py],exploitdb,https://github.com/offensive-security/exploitd...,720fabd0666423c25675eb8ff42161ec1264fd65,DB: 2020-07-28\n\n114 changes to exploits/shel...,Offensive Security,info@exploit-db.com,2020-07-28 05:01:59+00:00,Offensive Security,info@exploit-db.com,2020-07-28 05:01:59+00:00
6886,CVE-2020-3452,2020-07-29 05:02:05+00:00,CVE,[exploits/hardware/webapps/48722.txt],exploitdb,https://github.com/offensive-security/exploitd...,d8411b66138d6889b42b49e6375778b557da18be,DB: 2020-07-29\n\n1 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2020-07-29 05:02:05+00:00,Offensive Security,info@exploit-db.com,2020-07-29 05:02:05+00:00
6870,CVE-2020-15038,2020-07-30 05:02:03+00:00,CVE,[exploits/php/webapps/48724.txt],exploitdb,https://github.com/offensive-security/exploitd...,8a30306a81feb1919530762184633cb4ff84fc7d,DB: 2020-07-30\n\n2 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2020-07-30 05:02:03+00:00,Offensive Security,info@exploit-db.com,2020-07-30 05:02:03+00:00


In [250]:
df.to_csv("../output/all.csv")

cve = df[df['reftype'] == "CVE"]
cve.to_csv("../output/cve.csv")

In [251]:
cve

,reference,min_date,reftype,fpath,project_name,data_source,hash,msg,author_name,author_email,author_date,committer_name,committer_email,committer_date
2217,CVE-2011-0364,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/17155.py],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2335,CVE-2011-3167,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/18388.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2336,CVE-2011-3175,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/19958.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2337,CVE-2011-3176,2013-12-03 19:44:07+00:00,CVE,[platforms/windows/remote/19959.rb],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
2338,CVE-2011-3192,2013-12-03 19:44:07+00:00,CVE,[platforms/linux/dos/18221.c],exploitdb,https://github.com/offensive-security/exploitd...,fffbf04102c4588cace1156d28a7adc3da3f8af4,Updated,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00,Offensive Security,info@exploit-db.com,2013-12-03 19:44:07+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6834,CVE-2020-11749,2020-07-27 05:02:04+00:00,CVE,[exploits/php/webapps/48707.txt],exploitdb,https://github.com/offensive-security/exploitd...,e46d9f65ff00b5da8af2bf715f59767ae265523c,DB: 2020-07-27\n\n32 changes to exploits/shell...,Offensive Security,info@exploit-db.com,2020-07-27 05:02:04+00:00,Offensive Security,info@exploit-db.com,2020-07-27 05:02:04+00:00
6488,CVE-2019-16116,2020-07-28 05:01:59+00:00,CVE,[exploits/windows/remote/48657.py],exploitdb,https://github.com/offensive-security/exploitd...,720fabd0666423c25675eb8ff42161ec1264fd65,DB: 2020-07-28\n\n114 changes to exploits/shel...,Offensive Security,info@exploit-db.com,2020-07-28 05:01:59+00:00,Offensive Security,info@exploit-db.com,2020-07-28 05:01:59+00:00
6886,CVE-2020-3452,2020-07-29 05:02:05+00:00,CVE,[exploits/hardware/webapps/48722.txt],exploitdb,https://github.com/offensive-security/exploitd...,d8411b66138d6889b42b49e6375778b557da18be,DB: 2020-07-29\n\n1 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2020-07-29 05:02:05+00:00,Offensive Security,info@exploit-db.com,2020-07-29 05:02:05+00:00
6870,CVE-2020-15038,2020-07-30 05:02:03+00:00,CVE,[exploits/php/webapps/48724.txt],exploitdb,https://github.com/offensive-security/exploitd...,8a30306a81feb1919530762184633cb4ff84fc7d,DB: 2020-07-30\n\n2 changes to exploits/shellc...,Offensive Security,info@exploit-db.com,2020-07-30 05:02:03+00:00,Offensive Security,info@exploit-db.com,2020-07-30 05:02:03+00:00
